In [ ]:
#hide
import sys, os
from pathlib import Path

# Insert in Path Project Directory
sys.path.insert(0, str(Path().cwd().parent))

%load_ext autoreload
%load_ext line_profiler
%load_ext cython
%autoreload 2 

In [ ]:
#exporti
import os
from pathlib import Path
from typing import *
from collections import defaultdict, namedtuple
from fastcore.basics import partialler, listify
from fastcore.utils import parallel
from fastcore.foundation import L
from rfpye.constants import BYTES_HEADER, ENDMARKER, KEY_ATTRS
from rfpye.blocks import MAIN_BLOCKS
from rfpye.utils import get_files, getattrs, bin2int, bin2str
from rfpye.cyparser import cy_extract_compressed
from loguru import logger
import pandas as pd
import numpy as np

In [ ]:
%%cython --annotate
cimport cython

ctypedef object CrfsGPS

@cython.boundscheck(False)
@cython.wraparound(False)
cpdef dict classify_blocks(list byte_blocks):
    """Receives an iterable with binary blocks and returns a dict with the metadata from file, the gps class and a list with the different spectrum classes"""
    
    cdef dict meta = {}
    cdef dict spec = {}
    cdef object gps
    cdef list gps_attrs
    gps = CrfsGPS()
    gps_attrs = [f'_{k}' for k in BLOCK_ATTRS.get(40, [])]
    for byte_block in byte_blocks:
        block = create_block(byte_block)
        if not block: continue 
        if block.type  == 40:
            for k in BLOCK_ATTRS.get(40, []):
                getattr(gps, f'_{k}').append(getattr(block, k))
            continue
        attrs=KEY_ATTRS.get(block.type, [])
        values = getattrs(block, attrs=attrs)
        if block.type in SPECTRAL_BLOCKS:
            if values not in fluxos:
                attrs = [a for a in attrs] + ['thresh', 'minimum']
                metadata = namedtuple('metadata', attrs)
                attrs = list(values) + [block.thresh, block.minimum]
                spec[values] = CrfsSpectrum(metadata(*attrs))
            time = getattr(block, 'wallclock_datetime')
            attr = 'raw_data' if block.type in (64, 68) else 'levels'
            data = getattr(block, attr)
            spec[values].append(time, data)
        else:
            meta.update(dict(zip(attrs, values)))
    meta['gps'] = gps
    meta['spectrum'] = list(spec.values()) 
    return meta